# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 17:59:44.468000 2622508 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 17:59:44.468000 2622508 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 17:59:45] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35718, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=10195

[2025-08-14 17:59:46] Using default HuggingFace chat template with detected content format: string


W0814 17:59:53.527000 2623380 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 17:59:53.527000 2623380 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 17:59:53.575000 2623379 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 17:59:53.575000 2623379 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 17:59:54] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-14 17:59:54] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=32.50 GB):  50%|█████     | 2/4 [00:05<00:05,  2.54s/it]

Capturing batches (bs=1 avail_mem=32.50 GB): 100%|██████████| 4/4 [00:09<00:00,  2.44s/it]
[2025-08-14 18:00:16] Capture draft cuda graph end. Time elapsed: 10.61 s. mem usage=0.27 GB. avail mem=32.44 GB.
[2025-08-14 18:00:16] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=32.44 GB


Capturing batches (bs=1 avail_mem=53.05 GB): 100%|██████████| 4/4 [00:00<00:00, 108.95it/s]
[2025-08-14 18:00:17] Capture draft extend cuda graph end. Time elapsed: 0.84 s. mem usage=-20.61 GB. avail mem=53.05 GB.
[2025-08-14 18:00:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.05 GB
[2025-08-14 18:00:17] INFO:     Started server process [2622508]
[2025-08-14 18:00:17] INFO:     Waiting for application startup.
[2025-08-14 18:00:17] INFO:     Application startup complete.
[2025-08-14 18:00:17] INFO:     Uvicorn running on http://127.0.0.1:35718 (Press CTRL+C to quit)


[2025-08-14 18:00:18] INFO:     127.0.0.1:43660 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-14 18:00:18] INFO:     127.0.0.1:43662 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 18:00:18] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 18:00:18] INFO:     127.0.0.1:43670 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 18:00:18] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 18:00:23] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 18:00:23] INFO:     127.0.0.1:58960 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 18:00:30.079000 2626687 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:00:30.079000 2626687 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:00:32] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32738, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=6509168

[2025-08-14 18:00:33] Using default HuggingFace chat template with detected content format: string


W0814 18:00:40.201000 2627383 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:00:40.201000 2627383 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0814 18:00:40.802000 2627382 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:00:40.802000 2627382 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=36.25 GB): 100%|██████████| 4/4 [00:17<00:00,  4.39s/it]
[2025-08-14 18:01:46] Capture draft cuda graph end. Time elapsed: 18.27 s. mem usage=-18.98 GB. avail mem=36.20 GB.
[2025-08-14 18:01:46] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=36.20 GB


Capturing batches (bs=1 avail_mem=36.10 GB): 100%|██████████| 4/4 [00:00<00:00, 45.04it/s]
[2025-08-14 18:01:46] Capture draft extend cuda graph end. Time elapsed: 0.84 s. mem usage=0.10 GB. avail mem=36.10 GB.
[2025-08-14 18:01:46] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=36.10 GB


[2025-08-14 18:01:47] INFO:     Started server process [2626687]
[2025-08-14 18:01:47] INFO:     Waiting for application startup.
[2025-08-14 18:01:47] INFO:     Application startup complete.
[2025-08-14 18:01:47] INFO:     Uvicorn running on http://127.0.0.1:32738 (Press CTRL+C to quit)
[2025-08-14 18:01:47] INFO:     127.0.0.1:56200 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 18:01:48] INFO:     127.0.0.1:56208 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 18:01:48] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 18:01:50] INFO:     127.0.0.1:56224 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 18:01:50] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 18:01:52] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 18:01:52] INFO:     127.0.0.1:56226 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 18:01:59.004000 2636330 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:01:59.004000 2636330 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:02:00] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38459, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-14 18:02:01] Using default HuggingFace chat template with detected content format: string


W0814 18:02:08.182000 2636561 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:02:08.182000 2636561 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 18:02:08.183000 2636560 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:02:08.183000 2636560 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:02:09] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=57.88 GB): 100%|██████████| 4/4 [00:06<00:00,  1.60s/it]
[2025-08-14 18:02:37] Capture draft cuda graph end. Time elapsed: 7.02 s. mem usage=-9.12 GB. avail mem=50.49 GB.
[2025-08-14 18:02:37] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=50.49 GB


Capturing batches (bs=1 avail_mem=50.39 GB): 100%|██████████| 4/4 [00:00<00:00, 93.05it/s]
[2025-08-14 18:02:38] Capture draft extend cuda graph end. Time elapsed: 0.67 s. mem usage=0.10 GB. avail mem=50.39 GB.
[2025-08-14 18:02:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=50.39 GB


[2025-08-14 18:02:38] INFO:     Started server process [2636330]
[2025-08-14 18:02:38] INFO:     Waiting for application startup.
[2025-08-14 18:02:38] INFO:     Application startup complete.
[2025-08-14 18:02:38] INFO:     Uvicorn running on http://127.0.0.1:38459 (Press CTRL+C to quit)


[2025-08-14 18:02:38] INFO:     127.0.0.1:41662 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 18:02:39] INFO:     127.0.0.1:41666 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 18:02:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 18:02:39] INFO:     127.0.0.1:41672 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 18:02:39] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 18:02:44] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 18:02:44] INFO:     127.0.0.1:56650 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 18:02:50.775000 2639477 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:02:50.775000 2639477 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:02:52] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32763, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-14 18:02:53] Using default HuggingFace chat template with detected content format: string


W0814 18:02:59.423000 2640155 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:02:59.423000 2640155 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0814 18:02:59.783000 2640154 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:02:59.783000 2640154 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:04<00:00,  1.23s/it]
[2025-08-14 18:03:31] Capture draft cuda graph end. Time elapsed: 5.45 s. mem usage=0.37 GB. avail mem=57.69 GB.
[2025-08-14 18:03:31] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=57.69 GB


Capturing batches (bs=1 avail_mem=53.98 GB): 100%|██████████| 4/4 [00:00<00:00, 94.80it/s]
[2025-08-14 18:03:31] Capture draft extend cuda graph end. Time elapsed: 0.58 s. mem usage=3.71 GB. avail mem=53.98 GB.
[2025-08-14 18:03:31] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=53.98 GB


[2025-08-14 18:03:32] INFO:     Started server process [2639477]
[2025-08-14 18:03:32] INFO:     Waiting for application startup.
[2025-08-14 18:03:32] INFO:     Application startup complete.
[2025-08-14 18:03:32] INFO:     Uvicorn running on http://127.0.0.1:32763 (Press CTRL+C to quit)


[2025-08-14 18:03:33] INFO:     127.0.0.1:58112 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-14 18:03:33] INFO:     127.0.0.1:58124 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 18:03:33] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 18:03:33] INFO:     127.0.0.1:58128 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 18:03:33] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 18:03:38] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 18:03:38] INFO:     127.0.0.1:58130 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 18:03:44.902000 2643985 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:03:44.902000 2643985 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:03:46] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32349, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=831829032, constrained_jso

[2025-08-14 18:03:46] Using default HuggingFace chat template with detected content format: string


W0814 18:03:54.126000 2644694 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:03:54.126000 2644694 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 18:03:54.126000 2644693 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 18:03:54.126000 2644693 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 18:03:55] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-14 18:03:55] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=58.94 GB):  50%|█████     | 2/4 [00:01<00:01,  1.13it/s]

Capturing batches (bs=1 avail_mem=58.94 GB): 100%|██████████| 4/4 [00:02<00:00,  1.67it/s]
[2025-08-14 18:04:09] Capture draft cuda graph end. Time elapsed: 3.22 s. mem usage=0.01 GB. avail mem=58.94 GB.
[2025-08-14 18:04:09] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.94 GB


Capturing batches (bs=1 avail_mem=57.66 GB): 100%|██████████| 4/4 [00:00<00:00, 55.40it/s]
[2025-08-14 18:04:10] Capture draft extend cuda graph end. Time elapsed: 0.56 s. mem usage=1.28 GB. avail mem=57.66 GB.
[2025-08-14 18:04:10] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=57.66 GB


[2025-08-14 18:04:10] INFO:     Started server process [2643985]
[2025-08-14 18:04:10] INFO:     Waiting for application startup.
[2025-08-14 18:04:10] INFO:     Application startup complete.
[2025-08-14 18:04:10] INFO:     Uvicorn running on http://0.0.0.0:32349 (Press CTRL+C to quit)
[2025-08-14 18:04:10] INFO:     127.0.0.1:42254 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 18:04:11] INFO:     127.0.0.1:42266 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 18:04:11] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-14 18:05:01] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-14 18:05:02] INFO:     127.0.0.1:42276 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 18:05:02] The server is fired up and ready to roll!


[2025-08-14 18:05:02] Decode batch. #running-req: 1, #token: 99, token usage: 0.00, accept len: 1.78, cuda graph: True, gen throughput (token/s): 1.53, #queue-req: 0, 


[2025-08-14 18:05:02] Decode batch. #running-req: 1, #token: 166, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 150.92, #queue-req: 0, 


[2025-08-14 18:05:03] Decode batch. #running-req: 1, #token: 240, token usage: 0.01, accept len: 1.85, cuda graph: True, gen throughput (token/s): 167.55, #queue-req: 0, 


[2025-08-14 18:05:03] Decode batch. #running-req: 1, #token: 307, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 152.69, #queue-req: 0, 


[2025-08-14 18:05:04] Decode batch. #running-req: 1, #token: 370, token usage: 0.02, accept len: 1.57, cuda graph: True, gen throughput (token/s): 138.42, #queue-req: 0, 


[2025-08-14 18:05:04] Decode batch. #running-req: 1, #token: 441, token usage: 0.02, accept len: 1.77, cuda graph: True, gen throughput (token/s): 160.74, #queue-req: 0, 


[2025-08-14 18:05:05] Decode batch. #running-req: 1, #token: 510, token usage: 0.02, accept len: 1.73, cuda graph: True, gen throughput (token/s): 161.03, #queue-req: 0, 
[2025-08-14 18:05:05] INFO:     127.0.0.1:37006 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).